# Info
Phase Shift Full Bridge Converter Design tool for battery charger.
Design for variable output voltage with limited maximum output current and power.

# Input Data

In [1]:
import math
import numpy as np

#Input DC range
Vin_min=390
Vin_max=410
Vin_nom=400

#Output voltage range
Vout_min=172 
#Vout_nom=264.7
Vout_max=300
#Vout_ripple=150e-3

#Output power
Pout_max=900

#Output current limit
Iout_max=3.4

#Estimated efficiency
#eff=0.96

#switching frequency (on the secondary side, seen by the output inductor)
#note the transformer is switched in fsw/2
fsw=300e3

#maximum duty cycle
#Note: Provide the limit lower than PWM IC max limit to have some margin. COncider that DC_max limit is also affected by value of resonant inductance + transfomrer primary leakage inductance
DC_max=0.8

#### Output Inductor ####
#inductor peak to peak ripple current (in A)
dIL_pp=1.02

#### Resonant Inductor ####
Lr = 15e-6

#### Transformer ####
#Estimation of transformer parasitic parameters
#Primary winding capacitance
Cp = 100e-12
#Primary leakage inductance
L_leak = 4e-6

Vsec_k=1.24

#### MOSFET Parameters ####
# VDS derating
kM=0.85
# WC rdson at 110degC
M_rdson=0.14
# Energy related Coss in F at Vin_max
Coss_ef = 85e-12

#### Output Diodes Parameters ####
# Vbr derating
kD=0.6

# Transformer Requirements

## Turns Ratio (1:Ns)

In [2]:
Vsec=Vsec_k*Vout_max/DC_max
N=Vsec/Vin_min
round(N,2)

1.19

# Worst-Case Effective Duty Cycle (-)

Note: The effective duty cycle is the duty cycle of the secondary voltage.

In [3]:
D_eff_max = (Vout_max/Vin_min)/N
round(D_eff_max,2)

0.65

# Output Inductor Requirements

## Required Inductance (in uH)

In [4]:
#inductor ripple current is the highest when D_eff=0.5, Vin_max. It analyze WC condition which leads to the highest inductor ripple current.
#For that condition, minimum required inductance is calculated.
Vout_test = Vin_max*0.5*N 
if Vout_test >= Vout_min and Vout_test <= Vout_max: #It tests if D_eff=0.5 is possible
    D_IL_calc = 0.5
    Vout_IL_calc = Vin_max*D_IL_calc*N
    
#When D_eff=0.5 leads to Vout out of the specified range it find D_eff closest to 0.5.    
elif Vout_test <= Vout_min: 
    D_IL_calc = (Vout_min/Vin_max)/N
    Vout_IL_calc = Vin_max*D_IL_calc*N
elif Vout_test >= Vout_max:
    D_IL_calc = (Vout_max/Vin_max)/N
    Vout_IL_calc = Vin_max*D_IL_calc*N

Vsec_IL_calc = Vin_max*N
#L_out = ((Vsec_IL_calc-Vout_IL_calc)*D_IL_calc*1/fsw+Vout_IL_calc*(1-D_IL_calc)*1/fsw)/(2*dIL*np.clip(Pout_max/Vout_IL_calc,0,Iout_max))
L_out = ((Vsec_IL_calc-Vout_IL_calc)*D_IL_calc*1/fsw+Vout_IL_calc*(1-D_IL_calc)*1/fsw)/(2*dIL_pp)
print(round(L_out*1e6,2))


399.38


# Resonant Inductor

## Maximum Duty-Cycle loss

In [14]:
dDC_max=DC_max-D_eff_max
print(round(dDC_max,2))

0.15


## Estimated Maximum Inductance of Resonant Inductor (in uH)

In [15]:
Lr_max = dDC_max*Vin_min/(N*Iout_max*2*fsw)-L_leak
print(round(Lr_max*1e6,2))

20.83


# ZVS Range

## Required energy to achieve ZVS (in uJ)

In [6]:
E_TR = 0.5* Cp*Vin_max*Vin_max
E_Coss = (0.5*Coss_ef*Vin_max*Vin_max)*2
E_ZVS=E_TR+E_Coss
print(round(E_ZVS*1e6,2))

22.69
